In [1]:
import numpy as np
import pandas as pd
import yahooquery as yq
import yfinance as yf
     
from functools import reduce

In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

columns_to_remove = ["Headquarters Location", "Date added", "CIK", "Founded"]
tickers = tickers.drop(columns=columns_to_remove)

tickers = tickers.rename(columns={"Security": "Full Name"})

tickers

,Symbol,Full Name,GICS Sector,GICS Sub-Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Biotechnology
4,ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment


## Data collection through each symbol via Yahoo Finance

In [3]:
# Iteratalle through each symbol in the DataFrame
yq_data = pd.DataFrame([])

for ticker in tickers['Symbol']:
    try:

      # Use yahooquery to fetch ESG data
      ticker_yq = yq.Ticker(ticker).esg_scores

      ticker_yq[ticker]["Symbol"] = ticker

      ticker_yq = pd.DataFrame([ticker_yq[ticker]])
      yq_data = pd.concat([yq_data, ticker_yq], ignore_index=True)

    except Exception as e:
      # print(e)
      continue

C:\Users\Chandu\AppData\Local\Temp\ipykernel_41176\2413634088.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  yq_data = pd.concat([yq_data, ticker_yq], ignore_index=True)


In [4]:
yq_data.head()

,maxAge,totalEsg,environmentScore,socialScore,governanceScore,ratingYear,ratingMonth,highestControversy,peerCount,esgPerformance,...,furLeather,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,Symbol
0,86400,42.86,19.53,16.82,6.51,2025,2,5.0,63,LEAD_PERF,...,False,False,False,False,False,False,False,False,False,MMM
1,86400,26.82,6.65,13.55,6.61,2025,2,1.0,269,AVG_PERF,...,False,False,False,False,False,False,False,False,False,AOS
2,86400,21.62,1.81,13.77,6.05,2025,2,3.0,306,AVG_PERF,...,False,False,False,False,False,False,False,False,False,ABT
3,86400,25.97,1.78,15.71,8.48,2025,2,2.0,504,AVG_PERF,...,False,False,False,False,False,False,False,False,False,ABBV
4,86400,11.09,0.14,6.94,4.01,2025,2,1.0,603,LAG_PERF,...,False,False,False,False,False,False,False,False,False,ACN


In [5]:
yq_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   maxAge                             494 non-null    int64  
 1   totalEsg                           494 non-null    float64
 2   environmentScore                   492 non-null    float64
 3   socialScore                        492 non-null    float64
 4   governanceScore                    492 non-null    float64
 5   ratingYear                         494 non-null    int64  
 6   ratingMonth                        494 non-null    int64  
 7   highestControversy                 494 non-null    float64
 8   peerCount                          494 non-null    int64  
 9   esgPerformance                     494 non-null    object 
 10  peerGroup                          494 non-null    object 
 11  relatedControversy                 494 non-null    object 

In [6]:
required_cols = ['Symbol', 'environmentScore', 'socialScore', 'governanceScore', 'totalEsg', 'highestControversy', 'percentile', 'ratingYear', 'ratingMonth']

In [7]:
yq_data.columns.name = ''
yq_data = yq_data[required_cols]
yq_data = yq_data.apply(pd.to_numeric, errors="ignore")

C:\Users\Chandu\AppData\Local\Temp\ipykernel_41176\3989346021.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  yq_data = yq_data.apply(pd.to_numeric, errors="ignore")


In [8]:
yq_data

,Symbol,environmentScore,socialScore,governanceScore,totalEsg,highestControversy,percentile,ratingYear,ratingMonth
0,MMM,19.53,16.82,6.51,42.86,5.0,NaN,2025,2
1,AOS,6.65,13.55,6.61,26.82,1.0,NaN,2025,2
2,ABT,1.81,13.77,6.05,21.62,3.0,NaN,2025,2
3,ABBV,1.78,15.71,8.48,25.97,2.0,NaN,2025,2
4,ACN,0.14,6.94,4.01,11.09,1.0,NaN,2025,2
...,...,...,...,...,...,...,...,...,...
489,XYL,10.49,8.01,4.36,22.86,1.0,NaN,2025,2
490,YUM,7.04,10.25,3.22,20.51,2.0,NaN,2025,2
491,ZBRA,1.79,3.34,4.81,9.94,0.0,NaN,2025,2
492,ZBH,4.89,13.99,7.31,26.19,2.0,NaN,2025,2


In [9]:
# Iterate through each symbol in the DataFrame
yf_data = pd.DataFrame([])

for ticker in tickers['Symbol']:
    try:

      # Use yfinance to fetch data for each symbol
      ticker_info = yf.Ticker(ticker).info

      # Create a DataFrame for the current symbol's data
      ticker_data = pd.DataFrame({
        'Symbol': [ticker],
        'marketCap': [ticker_info.get('marketCap')],
        'beta': [ticker_info.get('beta')],
        'overallRisk': [ticker_info.get('overallRisk')]
      })

      yf_data = pd.concat([yf_data, ticker_data], ignore_index=True)

    except Exception as e:
      # print(e)
      continue

C:\Users\Chandu\AppData\Local\Temp\ipykernel_41176\2684153154.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  yf_data = pd.concat([yf_data, ticker_data], ignore_index=True)


In [10]:
yf_data

,Symbol,marketCap,beta,overallRisk
0,MMM,79773720576,1.017,9
1,AOS,9792984064,1.177,10
2,ABT,234271948800,0.693,6
3,ABBV,372047511552,0.598,7
4,ACN,213814083584,1.239,2
...,...,...,...,...
498,XYL,31343214592,1.081,2
499,YUM,44449783808,1.029,3
500,ZBRA,15134257152,1.679,10
501,ZBH,20965316608,1.039,2


In [11]:
sp500_esg_data = reduce(lambda x,y: pd.merge(x,y, on='Symbol', how='outer'), [tickers, yq_data, yf_data])
sp500_esg_data

,Symbol,Full Name,GICS Sector,GICS Sub-Industry,environmentScore,socialScore,governanceScore,totalEsg,highestControversy,percentile,ratingYear,ratingMonth,marketCap,beta,overallRisk
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,1.14,6.23,3.93,11.31,1.0,NaN,2025.0,2.0,36079788032,1.111,7
1,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",0.55,7.31,8.50,16.36,3.0,NaN,2025.0,2.0,3535151038464,1.178,1
2,ABBV,AbbVie,Health Care,Biotechnology,1.78,15.71,8.48,25.97,2.0,NaN,2025.0,2.0,372047511552,0.598,7
3,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines",1.30,13.81,8.28,23.39,2.0,NaN,2025.0,2.0,83738001408,1.108,10
4,ABT,Abbott Laboratories,Health Care,Health Care Equipment,1.81,13.77,6.05,21.62,3.0,NaN,2025.0,2.0,234271948800,0.693,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,10.49,8.01,4.36,22.86,1.0,NaN,2025.0,2.0,31343214592,1.081,2
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,7.04,10.25,3.22,20.51,2.0,NaN,2025.0,2.0,44449783808,1.029,3
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,4.89,13.99,7.31,26.19,2.0,NaN,2025.0,2.0,20965316608,1.039,2
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,1.79,3.34,4.81,9.94,0.0,NaN,2025.0,2.0,15134257152,1.679,10


In [12]:
sp500_esg_data.isnull().sum()

Symbol                  0
Full Name               0
GICS Sector             0
GICS Sub-Industry       0
environmentScore       11
socialScore            11
governanceScore        11
totalEsg                9
highestControversy      9
percentile            500
ratingYear              9
ratingMonth             9
marketCap               2
beta                   10
overallRisk             9
dtype: int64

In [13]:
sp500_esg_data.drop(columns = ['percentile'], inplace = True)

In [14]:
sp500_esg_data.isnull().sum()

Symbol                 0
Full Name              0
GICS Sector            0
GICS Sub-Industry      0
environmentScore      11
socialScore           11
governanceScore       11
totalEsg               9
highestControversy     9
ratingYear             9
ratingMonth            9
marketCap              2
beta                  10
overallRisk            9
dtype: int64

In [15]:
sp500_esg_data = sp500_esg_data.dropna()
sp500_esg_data.isnull().sum()

Symbol                0
Full Name             0
GICS Sector           0
GICS Sub-Industry     0
environmentScore      0
socialScore           0
governanceScore       0
totalEsg              0
highestControversy    0
ratingYear            0
ratingMonth           0
marketCap             0
beta                  0
overallRisk           0
dtype: int64

### Creation of S&P 500 Constituents Price List

In [16]:
valid_symbols = sp500_esg_data['Symbol'].to_list()

In [17]:
start_date = "2023-12-31"
end_date = "2025-03-01"

In [18]:
sp500_price_data = yf.download(valid_symbols, start=start_date, end=end_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  484 of 484 completed

1 Failed download:
['HWM']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-31 -> 2025-03-01)')


In [19]:
sp500_price_data

Price      Adj Close       Close                                      \
Ticker           HWM           A        AAPL        ABBV        ABNB   
Date                                                                   
2024-01-02       NaN  137.793900  184.532089  152.743393  134.479996   
2024-01-03       NaN  130.256195  183.150391  153.355057  133.419998   
2024-01-04       NaN  130.097305  180.824371  154.310791  133.720001   
2024-01-05       NaN  129.660324  180.098694  154.960663  135.979996   
2024-01-08       NaN  132.460907  184.452545  154.282104  140.080002   
...              ...         ...         ...         ...         ...   
2025-02-24       NaN  135.279999  247.100006  204.080002  144.820007   
2025-02-25       NaN  134.179993  247.039993  204.139999  141.550003   
2025-02-26       NaN  134.470001  240.360001  203.009995  144.029999   
2025-02-27       NaN  127.080002  237.300003  205.020004  139.449997   
2025-02-28       NaN  127.919998  241.839996  209.029999  138.869995   

Price                                                                  ...  \
Ticker             ABT       ACGL         ACN        ADBE         ADI  ...   
Date                                                                   ...   
2024-01-02  107.138481  71.840385  341.431366  580.070007  189.526352  ...   
2024-01-03  106.816628  73.162132  332.573761  571.789978  185.003098  ...   
2024-01-04  108.240593  73.666107  331.756866  567.049988  182.173615  ...   
2024-01-05  108.065041  73.038513  331.294312  564.599976  182.643570  ...   
2024-01-08  109.625534  73.333298  334.965271  580.549988  185.022675  ...   
...                ...        ...         ...         ...         ...  ...   
2025-02-24  134.949997  89.550003  363.910004  444.420013  235.963364  ...   
2025-02-25  135.820007  90.349998  362.950012  443.410004  234.280685  ...   
2025-02-26  135.960007  89.570000  358.350006  441.500000  232.717484  ...   
2025-02-27  135.869995  92.099998  356.869995  437.190002  222.880280  ...   
2025-02-28  138.009995  92.910004  348.500000  438.559998  229.063370  ...   

Price       Volume                                                         \
Ticker         WTW       WY     WYNN      XEL       XOM      XYL      YUM   
Date                                                                        
2024-01-02  394600  2317100  3013600  4627600  23483000  1089400  1485300   
2024-01-03  446200  2939800  2369100  4538600  23490800  1182800  1483600   
2024-01-04  273400  3317500  1694100  3129300  19395200  1352600  3019000   
2024-01-05  414700  4199900  2534700  3531000  15825000  1144300  1408800   
2024-01-08  308300  2262500  1441700  3738900  23370100   904300  1204100   
...            ...      ...      ...      ...       ...      ...      ...   
2025-02-24  614400  2669400  2160600  3010300  13153500  1511700  2120600   
2025-02-25  702500  4446600  2079800  3713400  14497000  2319200  2817400   
2025-02-26  700200  2097300  2241800  3907500  10446400  1280800  2112000   
2025-02-27  736200  3217200  2245100  4553500  15293200  2093200  1354200   
2025-02-28  547600  5766800  1468200  7085500  18821700  2510800  3003400   

Price                                 
Ticker          ZBH    ZBRA      ZTS  
Date                                  
2024-01-02  1087400  452900  1642300  
2024-01-03  2325500  404800  1493000  
2024-01-04  2099400  358100  1851900  
2024-01-05  1391000  293500  1088200  
2024-01-08  1427600  399500  1610600  
...             ...     ...      ...  
2025-02-24  1805300  517500  3698000  
2025-02-25  1679400  453200  4003300  
2025-02-26  1719400  572200  2609700  
2025-02-27  1259300  519400  2253500  
2025-02-28  2813600  611600  3333300  

[291 rows x 2421 columns]

In [20]:
sp500_price_data.columns

MultiIndex([('Adj Close',  'HWM'),
            (    'Close',    'A'),
            (    'Close', 'AAPL'),
            (    'Close', 'ABBV'),
            (    'Close', 'ABNB'),
            (    'Close',  'ABT'),
            (    'Close', 'ACGL'),
            (    'Close',  'ACN'),
            (    'Close', 'ADBE'),
            (    'Close',  'ADI'),
            ...
            (   'Volume',  'WTW'),
            (   'Volume',   'WY'),
            (   'Volume', 'WYNN'),
            (   'Volume',  'XEL'),
            (   'Volume',  'XOM'),
            (   'Volume',  'XYL'),
            (   'Volume',  'YUM'),
            (   'Volume',  'ZBH'),
            (   'Volume', 'ZBRA'),
            (   'Volume',  'ZTS')],
           names=['Price', 'Ticker'], length=2421)

In [22]:
import yfinance as yf
import pandas as pd

# Define stock symbols and date range
valid_symbols = sp500_esg_data['Symbol'].to_list()
start_date = "2023-12-31"
end_date = "2025-03-01"

# Download the full dataset
sp500_price_data = yf.download(valid_symbols, start=start_date, end=end_date)

# Filepath for saving
output_file = "sp500_price_data1.xlsx"

# Export each column to a separate sheet
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    for col in sp500_price_data.columns.levels[0]:  # Iterating over OHLCV columns
        sp500_price_data[col].to_excel(writer, sheet_name=col)  # Save each in a sheet

print(f"Data saved to {output_file}")


[*********************100%***********************]  484 of 484 completed

1 Failed download:
['HWM']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-31 -> 2025-03-01)')


Data saved to sp500_price_data1.xlsx


In [76]:
sp500_price_data = sp500_price_data["Close"][esg_rated_symbols]
sp500_price_data.head()

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,137.793900,184.532074,152.743393,134.479996,107.138481,71.840385,341.431335,580.070007,190.350952,69.449005,...,236.998962,33.775772,93.315514,61.121792,98.106003,112.081642,125.917305,120.354469,267.980011,194.055573
2024-01-03,130.256210,183.150391,153.355072,133.419998,106.816628,73.162132,332.573761,571.789978,185.808029,69.802177,...,237.670609,33.067051,92.753250,61.227722,98.930260,110.001122,125.985611,119.442314,252.520004,190.462143
2024-01-04,130.097321,180.824341,154.310806,133.720001,108.240593,73.666107,331.756897,567.049988,182.966217,68.484970,...,236.544632,32.610756,92.930801,61.372173,98.067665,110.760368,125.585518,119.174614,252.970001,191.557938
2024-01-05,129.660324,180.098709,154.960663,135.979996,108.065041,73.038513,331.294281,564.599976,183.438217,67.587753,...,235.932251,32.698124,94.351265,61.372173,98.364784,110.582893,125.243950,118.956497,252.690002,192.357590
2024-01-08,132.460907,184.452560,154.282104,140.080002,109.625542,73.333298,334.965271,580.549988,185.827698,67.826370,...,239.191666,33.057343,95.732254,61.227722,96.725845,111.273102,125.536705,120.929512,261.089996,193.640945


In [77]:
sp500_esg_data.to_csv('sp500_esg_data_latest.csv', index = False)
sp500_price_data.to_csv('sp500_price_data_latest.csv')